In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from collections import Counter
from tqdm.notebook import tqdm
from sklearn.feature_extraction.text import CountVectorizer
import transformers
from konlpy.tag import Mecab

In [56]:
# matplotlib 한글 패치
from matplotlib import font_manager, rc
import platform

if platform.system() == 'Windows':
    rc('font', family='Malgun Gothic')
elif platform.system() == 'Darwin': # Mac
    rc('font', family='AppleGothic')
else: #linux
    rc('font', family='NanumGothic')

In [57]:
# .. 상위 폴더
train_pd = pd.read_csv('../../../../data/train.csv')
train_pd

,id,source,sentence_1,sentence_2,label,binary-label
0,boostcamp-sts-v1-train-000,nsmc-sampled,스릴도있고 반전도 있고 여느 한국영화 쓰레기들하고는 차원이 다르네요~,"반전도 있고,사랑도 있고재미도있네요.",2.2,0.0
1,boostcamp-sts-v1-train-001,slack-rtt,앗 제가 접근권한이 없다고 뜹니다;;,"오, 액세스 권한이 없다고 합니다.",4.2,1.0
2,boostcamp-sts-v1-train-002,petition-sampled,주택청약조건 변경해주세요.,주택청약 무주택기준 변경해주세요.,2.4,0.0
3,boostcamp-sts-v1-train-003,slack-sampled,입사후 처음 대면으로 만나 반가웠습니다.,화상으로만 보다가 리얼로 만나니 정말 반가웠습니다.,3.0,1.0
4,boostcamp-sts-v1-train-004,slack-sampled,뿌듯뿌듯 하네요!!,꼬옥 실제로 한번 뵈어요 뿌뿌뿌~!~!,0.0,0.0
...,...,...,...,...,...,...
9319,boostcamp-sts-v1-train-9319,petition-sampled,교원능력개발평가에서 교원이 보호받을 수 있는 장치를 마련해야합니다,본인이 납부한 국민연금 금액을 기준으로 대출을 받을 수 있는 제도를 마련해 주세요,0.2,0.0
9320,boostcamp-sts-v1-train-9320,petition-sampled,여성가족부의 폐지를 원합니드,여성가족부 폐지를 청원 합니다.,4.2,1.0
9321,boostcamp-sts-v1-train-9321,petition-sampled,국회의원들 월급좀 줄여주세요,공무원 봉급좀 줄이지좀 마세요,0.6,0.0
9322,boostcamp-sts-v1-train-9322,slack-sampled,오늘 못한 점심은 다음에 다시 츄라이 하기로 해요!!,오늘 못먹은 밥은 꼭 담에 먹기로 하고요!!,3.2,1.0


In [58]:
# apply 함수로 일괄 적용

# sentence 합치기
train_pd['sentence_a'] = train_pd['sentence_1'] + ' ' + train_pd['sentence_2']

# 공백 기준으로 토큰 나누기
train_pd['tokens_a']=train_pd['sentence_a'].apply(lambda x: x.split())
train_pd['tokens_1']=train_pd['sentence_1'].apply(lambda x: x.split())
train_pd['tokens_2']=train_pd['sentence_2'].apply(lambda x: x.split())

# 문장 길이, 토큰 개수 칼럼 만들기
train_pd['sen_len_a']=train_pd['sentence_a'].apply(lambda x: len(x))
train_pd['sen_len_1']=train_pd['sentence_1'].apply(lambda x: len(x))
train_pd['sen_len_2']=train_pd['sentence_2'].apply(lambda x: len(x))

train_pd['tok_count_a']=train_pd['tokens_a'].apply(lambda x: len(x))
train_pd['tok_count_1']=train_pd['tokens_1'].apply(lambda x: len(x))
train_pd['tok_count_2']=train_pd['tokens_2'].apply(lambda x: len(x))

# label 단순화, 소숫점 버림
train_pd['simple_label'] = train_pd['label'].apply(lambda x: int(x))

train_pd.head()

,id,source,sentence_1,sentence_2,label,binary-label,sentence_a,tokens_a,tokens_1,tokens_2,sen_len_a,sen_len_1,sen_len_2,tok_count_a,tok_count_1,tok_count_2,simple_label
0,boostcamp-sts-v1-train-000,nsmc-sampled,스릴도있고 반전도 있고 여느 한국영화 쓰레기들하고는 차원이 다르네요~,"반전도 있고,사랑도 있고재미도있네요.",2.2,0.0,"스릴도있고 반전도 있고 여느 한국영화 쓰레기들하고는 차원이 다르네요~ 반전도 있고,...","[스릴도있고, 반전도, 있고, 여느, 한국영화, 쓰레기들하고는, 차원이, 다르네요~...","[스릴도있고, 반전도, 있고, 여느, 한국영화, 쓰레기들하고는, 차원이, 다르네요~]","[반전도, 있고,사랑도, 있고재미도있네요.]",59,38,20,11,8,3,2
1,boostcamp-sts-v1-train-001,slack-rtt,앗 제가 접근권한이 없다고 뜹니다;;,"오, 액세스 권한이 없다고 합니다.",4.2,1.0,"앗 제가 접근권한이 없다고 뜹니다;; 오, 액세스 권한이 없다고 합니다.","[앗, 제가, 접근권한이, 없다고, 뜹니다;;, 오,, 액세스, 권한이, 없다고, ...","[앗, 제가, 접근권한이, 없다고, 뜹니다;;]","[오,, 액세스, 권한이, 없다고, 합니다.]",40,20,19,10,5,5,4
2,boostcamp-sts-v1-train-002,petition-sampled,주택청약조건 변경해주세요.,주택청약 무주택기준 변경해주세요.,2.4,0.0,주택청약조건 변경해주세요. 주택청약 무주택기준 변경해주세요.,"[주택청약조건, 변경해주세요., 주택청약, 무주택기준, 변경해주세요.]","[주택청약조건, 변경해주세요.]","[주택청약, 무주택기준, 변경해주세요.]",33,14,18,5,2,3,2
3,boostcamp-sts-v1-train-003,slack-sampled,입사후 처음 대면으로 만나 반가웠습니다.,화상으로만 보다가 리얼로 만나니 정말 반가웠습니다.,3.0,1.0,입사후 처음 대면으로 만나 반가웠습니다. 화상으로만 보다가 리얼로 만나니 정말 반가...,"[입사후, 처음, 대면으로, 만나, 반가웠습니다., 화상으로만, 보다가, 리얼로, ...","[입사후, 처음, 대면으로, 만나, 반가웠습니다.]","[화상으로만, 보다가, 리얼로, 만나니, 정말, 반가웠습니다.]",51,22,28,11,5,6,3
4,boostcamp-sts-v1-train-004,slack-sampled,뿌듯뿌듯 하네요!!,꼬옥 실제로 한번 뵈어요 뿌뿌뿌~!~!,0.0,0.0,뿌듯뿌듯 하네요!! 꼬옥 실제로 한번 뵈어요 뿌뿌뿌~!~!,"[뿌듯뿌듯, 하네요!!, 꼬옥, 실제로, 한번, 뵈어요, 뿌뿌뿌~!~!]","[뿌듯뿌듯, 하네요!!]","[꼬옥, 실제로, 한번, 뵈어요, 뿌뿌뿌~!~!]",32,10,21,7,2,5,0


In [59]:
tokenizer160 = transformers.AutoTokenizer.from_pretrained('klue/roberta-small')
tokenizer512 = transformers.AutoTokenizer.from_pretrained('klue/roberta-small', max_length=512)

In [60]:
df = train_pd.drop(columns=['id'])
data160 = []
for idx, item in tqdm(df.iterrows(), desc='tokenizing', total=len(df)):
# 두 입력 문장을 [SEP] 토큰으로 이어붙여서 전처리합니다.
    text = '[SEP]'.join([item[text_column] for text_column in ['sentence_1', 'sentence_2']])
    outputs = tokenizer160(text, add_special_tokens=True, padding='max_length', max_length=160, truncation=True)
    data160.append(outputs['input_ids'])

tokenizing:   0%|          | 0/9324 [00:00<?, ?it/s]

In [61]:
df = train_pd.drop(columns=['id'])
data512 = []
for idx, item in tqdm(df.iterrows(), desc='tokenizing', total=len(df)):
# 두 입력 문장을 [SEP] 토큰으로 이어붙여서 전처리합니다.
    text = '[SEP]'.join([item[text_column] for text_column in ['sentence_1', 'sentence_2']])
    outputs = tokenizer160(text, add_special_tokens=True, padding='max_length', truncation=True)
    data512.append(outputs['input_ids'])

tokenizing:   0%|          | 0/9324 [00:00<?, ?it/s]

In [62]:
print(len(data160[0]))
print(data160[0])

160
[0, 15314, 2119, 2689, 2088, 8984, 2119, 1513, 2088, 11934, 3629, 16516, 6614, 2031, 19521, 2259, 4540, 2052, 4405, 2203, 2182, 97, 2, 8984, 2119, 1513, 2088, 16, 3784, 2119, 1513, 2088, 2070, 2044, 2119, 2689, 2203, 2182, 18, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [63]:
print(len(data512[0]))
print(data512[0][:160])

512
[0, 15314, 2119, 2689, 2088, 8984, 2119, 1513, 2088, 11934, 3629, 16516, 6614, 2031, 19521, 2259, 4540, 2052, 4405, 2203, 2182, 97, 2, 8984, 2119, 1513, 2088, 16, 3784, 2119, 1513, 2088, 2070, 2044, 2119, 2689, 2203, 2182, 18, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [64]:
train_pd[train_pd['sen_len_a']==max(train_pd['sen_len_a'])]

,id,source,sentence_1,sentence_2,label,binary-label,sentence_a,tokens_a,tokens_1,tokens_2,sen_len_a,sen_len_1,sen_len_2,tok_count_a,tok_count_1,tok_count_2,simple_label
2412,boostcamp-sts-v1-train-2412,nsmc-rtt,‥ 김민희를알수록 의심되는 정체. 완벽한연기. 완벽한몰입과스릴. 작은반전. 무엇보다...,‥ 김민희를 알면 알수록 의심스러워진다. 완벽한 연기. 완벽한 몰입과 스릴. 작은 ...,4.0,1.0,‥ 김민희를알수록 의심되는 정체. 완벽한연기. 완벽한몰입과스릴. 작은반전. 무엇보다...,"[‥, 김민희를알수록, 의심되는, 정체., 완벽한연기., 완벽한몰입과스릴., 작은반...","[‥, 김민희를알수록, 의심되는, 정체., 완벽한연기., 완벽한몰입과스릴., 작은반...","[‥, 김민희를, 알면, 알수록, 의심스러워진다., 완벽한, 연기., 완벽한, 몰입...",209,94,114,43,17,26,4


In [65]:
print(len(data160[2412]))
print(data160[0][:250])

160
[0, 15314, 2119, 2689, 2088, 8984, 2119, 1513, 2088, 11934, 3629, 16516, 6614, 2031, 19521, 2259, 4540, 2052, 4405, 2203, 2182, 97, 2, 8984, 2119, 1513, 2088, 16, 3784, 2119, 1513, 2088, 2070, 2044, 2119, 2689, 2203, 2182, 18, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [66]:
print(len(data512[2412]))
print(data512[0][:250])

512
[0, 15314, 2119, 2689, 2088, 8984, 2119, 1513, 2088, 11934, 3629, 16516, 6614, 2031, 19521, 2259, 4540, 2052, 4405, 2203, 2182, 97, 2, 8984, 2119, 1513, 2088, 16, 3784, 2119, 1513, 2088, 2070, 2044, 2119, 2689, 2203, 2182, 18, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


### Tokenizer max_length 자료 조사

- 우선, 실험적으로 max_length 변화에 차이가 존재하지 않았음
- Tokenzier 호출 시 max_length는 truncation/padding 의 maximum length를 control
- Padding에 들어간 'max_length'는 model의 최대 수용 가능 입력 길이를 의미하며 이것을 사용하면 해당 길이까지 padding 처리해줌
    - ![Tokenizer max_length Description](../../../src/max_length_d.png) 
    - ![Padding max_length Description](../../../src/padding_d.png)  
    - [Hugging Face tokenzier doc](https://huggingface.co/docs/transformers/main_classes/tokenizer)
    - [Hugging Face pad_trunc doc](https://huggingface.co/docs/transformers/pad_truncation)
    #
- 사용한 Tokenizer는 [BertTokenizer](https://huggingface.co/klue/roberta-small/blob/main/tokenizer_config.json)
- 아래는 관련 문서
    - [AutoTokenzier Source Code](https://huggingface.co/transformers/v3.0.2/model_doc/auto.html#autotokenizer)
    - [AutoConfig Source Code](https://huggingface.co/transformers/v3.0.2/model_doc/auto.html#autoconfig)
    - [BertConfig 문서](https://huggingface.co/transformers/v3.0.2/model_doc/bert.html#transformers.BertConfig)

In [73]:
tokenizer = transformers.AutoTokenizer.from_pretrained('klue/roberta-small')
df = train_pd.drop(columns=['id'])
data = []
for idx, item in tqdm(df.iterrows(), desc='tokenizing', total=len(df)):
# 두 입력 문장을 [SEP] 토큰으로 이어붙여서 전처리합니다.
    text = '[SEP]'.join([item[text_column] for text_column in ['sentence_1', 'sentence_2']])
    outputs = tokenizer(text, add_special_tokens=True, padding='do_not_pad', truncation=True)
    data.append(outputs['input_ids'])

tokenizing:   0%|          | 0/9324 [00:00<?, ?it/s]

In [76]:
max(list(map(lambda x : len(x), data)))

171

In [68]:
''.join(tokenizer.batch_decode(data[0][:160]))

'[CLS]스릴##도##있##고반전##도있##고여느한국##영화쓰레기##들##하고##는차원##이다르##네##요~[SEP]반전##도있##고,사랑##도있##고##재##미##도##있##네##요.[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]'

In [69]:
''.join(tokenizer.batch_decode(data[2412][:160]))

'[CLS]‥김민##희##를##알##수록의심##되##는정체.완벽##한##연##기.완벽##한##몰##입##과##스##릴.작##은##반##전.무엇##보##다현실##성이##있##는‥인간##의한계##와그걸벗어나##려##는욕심##을##보##여##준‥내보##기##엔완벽##한시나리오.[SEP]‥김민##희##를알##면알##수록의심##스##러워##진다.완벽##한연기.완벽##한몰입##과스릴.작##은회전.무엇##보##다현실##적인시나리오##다.인간##의한계##와그것##을벗어나##고##자하##는욕망##을보여준다.시나리오##가완벽##했##던것같##다.[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]'

In [16]:
train_pd['bert_tokenized']=pd.Series(map(lambda x : ''.join(tokenizer.batch_decode(x)),data))

In [17]:
train_pd[['sentence_1','sentence_2','bert_tokenized']]

,sentence_1,sentence_2,bert_tokenized
0,스릴도있고 반전도 있고 여느 한국영화 쓰레기들하고는 차원이 다르네요~,"반전도 있고,사랑도 있고재미도있네요.",[CLS]스릴##도##있##고반전##도있##고여느한국##영화쓰레기##들##하고##는...
1,앗 제가 접근권한이 없다고 뜹니다;;,"오, 액세스 권한이 없다고 합니다.","[CLS]앗제##가접근##권##한##이없##다고[UNK];;[SEP]오,액세##스권..."
2,주택청약조건 변경해주세요.,주택청약 무주택기준 변경해주세요.,[CLS]주택##청##약##조건변경##해##주##세요.[SEP]주택##청##약무주택...
3,입사후 처음 대면으로 만나 반가웠습니다.,화상으로만 보다가 리얼로 만나니 정말 반가웠습니다.,[CLS]입사##후처음대면##으로만나반가웠##습##니다.[SEP]화상##으로##만보...
4,뿌듯뿌듯 하네요!!,꼬옥 실제로 한번 뵈어요 뿌뿌뿌~!~!,[CLS]뿌듯##뿌##듯하##네##요!![SEP]꼬##옥실제로한번뵈##어요뿌##뿌#...
...,...,...,...
9319,교원능력개발평가에서 교원이 보호받을 수 있는 장치를 마련해야합니다,본인이 납부한 국민연금 금액을 기준으로 대출을 받을 수 있는 제도를 마련해 주세요,[CLS]교원##능력##개발##평가##에서교원##이보호##받##을수있##는장치##를...
9320,여성가족부의 폐지를 원합니드,여성가족부 폐지를 청원 합니다.,[CLS]여성##가족##부##의폐지##를원##합##니##드[SEP]여성##가족##부...
9321,국회의원들 월급좀 줄여주세요,공무원 봉급좀 줄이지좀 마세요,[CLS]국회의원##들월급##좀줄여##주##세요[SEP]공무원봉급##좀줄이##지##...
9322,오늘 못한 점심은 다음에 다시 츄라이 하기로 해요!!,오늘 못먹은 밥은 꼭 담에 먹기로 하고요!!,[CLS]오늘못한점심##은다음##에다시츄##라이하##기##로해요!![SEP]오늘못#...
